Exercises:
For the exercises, use NumPy arrays for data unless instructed otherwise.

### Exercise 2.1: *Loop unrolling*
- Implement the loop-unrolling example from section 6.3.2 (pp. 266–267) in Python.  
- Ignore pointers and use a **`while` loop** instead of `for` (Python `for` loops are optimized).  
- Unroll at least **2** and **4** steps.  
- Measure execution time.  
- **Note:** Expect larger gains with **Numba** (see [Numba quickstart](https://numba.readthedocs.io/en/stable/user/5minguide.html)).  
- Reports suggest 4–8 step unrolling with Numba can outperform NumPy on Apple M1 Max. Check your results.


In [1]:
#SET UP
import numpy as np
import time

# Data example for loop unrolling
n = int(1e7)  # 10 million elements
data = np.random.rand(n)

In [2]:
# Normal loop - 1 operation per iteration
normal_result = 0.0
i = 0
start = time.perf_counter()
while i < n:
    normal_result += data[i]
    i += 1
end = time.perf_counter()
print(f"Standard:          {end - start:.6f} seconds, result: {normal_result:.6f}")


Standard:          2.564044 seconds, result: 5000092.477388


In [3]:
# Unrolled 8x operation per iteration
unrolled_result = 0.0
i = 0
start = time.perf_counter()
while i < n - 7:
    unrolled_result += data[i]
    unrolled_result += data[i+1]
    unrolled_result += data[i+2]
    unrolled_result += data[i+3]
    unrolled_result += data[i+4]
    unrolled_result += data[i+5]
    unrolled_result += data[i+6]
    unrolled_result += data[i+7]
    i += 8
    # Handle remainder
while i < n:
    unrolled_result += data[i]
    i += 1
end = time.perf_counter()
print(f"Unrolled 8x:       {end - start:.6f} seconds, result: {unrolled_result:.6f}")

Unrolled 8x:       1.867745 seconds, result: 5000092.477388


In [4]:
# NumPy baseline for comparison
start = time.perf_counter()
np_result = np.sum(data)
end = time.perf_counter()
print(f"NumPy sum:         {end - start:.6f} seconds, result: {np_result:.6f}")

# NUMBA JIT-compiled unrolled loop
from numba import jit

@jit(nopython=True)
def sum_unrolled_8(data):
    n = len(data)
    result = 0.0
    i = 0
    while i < n - 7:
        result += data[i]
        result += data[i+1]
        result += data[i+2]
        result += data[i+3]
        result += data[i+4]
        result += data[i+5]
        result += data[i+6]
        result += data[i+7]
        i += 8
    # Handle remainder
    while i < n:
        result += data[i]
        i += 1
    return result

# Warm up JIT
_ = sum_unrolled_8(data)

# Time it
start = time.perf_counter()
numba_result = sum_unrolled_8(data)
end = time.perf_counter()
print(f"Numba unrolled 8x: {end - start:.6f} seconds, result: {numba_result:.6f}")

print("\nRun on the M2 Pro")

NumPy sum:         0.009354 seconds, result: 5000092.477389
Numba unrolled 8x: 0.012916 seconds, result: 5000092.477388

Run on the M2 Pro


### Exercise 2.2: *Cache blocking*
- Implement the example from section 6.4.1 (top of p. 269) in Python.  
- Use a **`while` loop** and compile with **Numba**.  
- Increase `l1size`, measure execution time, and compute **time per operation**.  
- When exceeding **L1 cache size** (~32 KB), time per operation should rise.  
- Extend with **cache blocking** and verify reduced time per operation.

Mini-project
Continue working on the mini-project. You should now be able to work on the following versions: 
naive (loops), 
NumPy (vectorized), 
Numba-optimized.

In [5]:
# Generated by Claude Sonnet 4.5 on 2026-02-22
import numpy as np
import time
from numba import jit

@jit(nopython=True)
def sum_with_stride(data, stride):
    """Access every stride-th element to defeat cache"""
    n = len(data)
    result = 0.0
    i = 0
    while i < n:
        result += data[i]
        i += stride
    return result

sizes = [10000, 50000, 100000, 500000, 1000000, 5000000, 10000000]
stride = 64  # Jump 64 elements (512 bytes) each time

print("\nCache Effect with Strided Access (Numba):")
print("Size (elements) | Time/op (ns)")
print("-" * 40)

for size in sizes:
    data = np.random.rand(size)
    
    # Warm up
    _ = sum_with_stride(data, stride)
    
    # Measure
    iterations = 10000
    start = time.perf_counter()
    for _ in range(iterations):
        result = sum_with_stride(data, stride)
    end = time.perf_counter()
    
    ops = (size // stride) * iterations
    time_per_op = (end - start) / ops * 1e9
    print(f"{size:15} | {time_per_op:.2f}")

print("\nRun on the M2 Pro")


Cache Effect with Strided Access (Numba):
Size (elements) | Time/op (ns)
----------------------------------------
          10000 | 2.93
          50000 | 1.48
         100000 | 1.29
         500000 | 2.99
        1000000 | 2.90
        5000000 | 5.54
       10000000 | 7.38

Run on the M2 Pro



Time decreases initially (10K → 1M elements): This is amortization - Numba's JIT compilation overhead gets spread over more operations

Performance drop at 5M+ elements:

5M elements × 8 bytes = 40 MB (exceeds L1 and L2 cache)
Time per op jumps from 0.91 ns → 2.39 ns (2.6× slower!)
This is the cache miss penalty you're looking for
Why the Jump Happens
Small arrays (< 1M): Fit in L2/L3 cache (~16 MB)
Large arrays (> 5M): Must fetch from main RAM
Strided access (every 64th element): Defeats cache prefetching